In [ ]:
!pip install redis --user

In [16]:
!pgrep redis-server

9521
60212


In [75]:
import redis

# db - номер базы (по умолчанию создаются 16)
pool = redis.ConnectionPool(host='localhost', port=26379, db=0)
r = redis.Redis(connection_pool=pool)
r

Redis<ConnectionPool<Connection<host=dev-usr-0017,port=26379,db=0>>>

In [18]:
# Сохранение множества пар ключ-значение
r.mset({"Eric":"Richmond", "Rusia":"Moscow"})

True

In [21]:
print(r.get("Eric")) # Возвращает байты
print(r.get("Eric").decode('utf-8')) # Возвращает строку

b'Richmond'
Richmond


In [25]:
# Инкрементация счетчика
print(r.set("count",1))
print(r.incr("count"))
print(r.get("count"))

True
2
b'2'


In [31]:
from  datetime import date

# Множества
#urls = {'site1','site2','site3'}
today = str(date.today())

r.sadd(today, 'site1','site2','site3','site1') # Добавляются только уникальные значения
r.smembers(today)

{b'site1', b'site2', b'site3'}

In [76]:
# Конвейрная обработка данных
import pickle

with open("/home/makilins/Downloads/movies.p","rb") as tmdb_file:
    data = pickle.load(tmdb_file)
    movs = data[0:10]
    json_docs = []
    for mov in movs:
        mov_short = {}
        mov_short["id"] = mov["id"]
        mov_short["title"] = mov["title"]
        mov_short["overview"] = mov["overview"]
        mov_short["tagline"] = mov["tagline"]
        mov_short["release_date"] = mov["release_date"]
        mov_short["budget"] = mov["budget"]
        mov_short["revenue"] = mov["revenue"]
        mov_short["vote_count"] = mov["vote_count"]
        mov_short["vote_average"] = mov["vote_average"]
        json_docs.append(mov_short)
        
json_docs

[{'budget': 220000000,
  'id': 24428,
  'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
  'release_date': '2012-05-04',
  'revenue': 1518594910,
  'tagline': 'Some assembly required.',
  'title': 'The Avengers',
  'vote_average': 7.2,
  'vote_count': 6535},
 {'budget': 160000000,
  'id': 27205,
  'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.',
  'release_date': '2010-07-16',
  'revenue': 825500000,
  'tagline': 'Your mind is the scene of the crime.',
  'title': 'Inception',
  'vot

In [77]:
# Конвейрная запись
with r.pipeline() as pipe:
    for mov in json_docs:
        pipe.hmset(mov['id'], mov)
    print(pipe.execute())

[True, True, True, True, True, True, True, True, True, True]


/opt/Anaconda3-4.4.0/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  after removing the cwd from sys.path.


In [78]:
# Асинхронная запись на диск
r.bgsave()

True

In [82]:
print(r.keys())
print(r.hgetall('24428'))
print(r.hget('24428','vote_count'))

[b'19995', b'68721', b'37724', b'49051', b'24428', b'70160', b'49026', b'27205', b'68718', b'155']
{b'id': b'24428', b'release_date': b'2012-05-04', b'tagline': b'Some assembly required.', b'vote_count': b'6539', b'vote_average': b'7.2', b'revenue': b'1518594910', b'budget': b'220000000', b'overview': b'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!', b'title': b'The Avengers'}
b'6539'


In [80]:
import logging
logging.basicConfig()


def vote_film(film_id):
    with r.pipeline() as pipe:
        error_count = 0
        error_limit = 3
        while True:
            if error_count > error_limit:
                break
            try:
                # Следим за изменениями объекта
                pipe.watch(film_id)
                # Проверяем наличие объекта
                vote_count = r.hget(film_id,'vote_count')
                if int(vote_count) > 0:
                    pipe.multi()
                    r.hincrby(film_id,'vote_count',1)
                    pipe.execute()
                    break
                else:
                    pipe.unwatch() # Останавливаем поиск объекта
                    raise Exception(film_id + " can't found.")
            except redis.WatchError:
                error_count += 1
                print("error", film_id, error_count)
                
    

In [81]:
vote_film('24428')

error 24428 1
error 24428 2
error 24428 3
error 24428 4


In [72]:
r.lastsave()

datetime.datetime(2020, 5, 16, 13, 39, 40)

## Публикация - подписка

In [83]:
pubsub = r.pubsub()
pubsub

In [85]:
pubsub.psubscribe("*-channel") # Подписаться на каналы по паттерну
pubsub.get_message() # получить сообщение

# channel - имя канала от которого получено сообщение
# data - данные в сообщении

{'channel': b'*-channel', 'data': 1, 'pattern': None, 'type': 'psubscribe'}

In [86]:
r.publish("first-channel", "some data") # Оправка сообщения
pubsub.get_message()

{'channel': b'*-channel', 'data': 1, 'pattern': None, 'type': 'psubscribe'}

In [87]:
pubsub.get_message()

{'channel': b'first-channel',
 'data': b'some data',
 'pattern': b'*-channel',
 'type': 'pmessage'}

In [93]:
# Регистрация обраотчика сообщений
def first_handler(message):
    print("MESSAGE DATA: ", message["data"])
    print("MESSAGE CHANNEL: ", message["channel"])

pubsub.subscribe(**{"first-channel": first_handler})
r.publish("first-channel", "some data2") # Оправка сообщения
pubsub.get_message()
pubsub.get_message()

MESSAGE DATA:  b'some data2'
MESSAGE CHANNEL:  b'first-channel'


In [96]:
message = pubsub.get_message()
message

In [103]:
pubsub2 = r.pubsub()
pubsub2.subscribe(**{"first-channel": first_handler})
r.publish("first-channel", "some data2") # Оправка сообщения
r.publish("first-channel", "some data3") # Оправка сообщения
r.publish("first-channel", "some data4") # Оправка сообщения

thread = pubsub2.run_in_thread(sleep_time=0.001) # Обработка сообщений в фоновом режиме в отдельном потоке
thread.stop()

In [107]:
# Можно итерироваться по данным в redis

for key in r.scan_iter():
    print(key, " ", r.hgetall(key))

b'49026'   {b'revenue': b'1081041287', b'id': b'49026', b'tagline': b'The Legend Ends', b'release_date': b'2012-07-20', b'overview': b"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.", b'budget': b'250000000', b'vote_count': b'4706', b'title': b'The Dark Knight Rises', b'vote_average': b'7.3'}
b'19995'   {b'revenue': b'2781505847', b'id': b'19995', b'tagline': b'Enter the World of Pandora.', b'release_date': b'2009-12-18', b'overview': b'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',